In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

24/06/17 08:46:22 WARN Utils: Your hostname, ast-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.29.20 instead (on interface wlp2s0)
24/06/17 08:46:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/17 08:46:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.functions import *

In [12]:
orders_schema = """order_id long, customer_id long, customer_fname string, customer_lname string, 
                    city string, state string, pincode long,
                    line_items array<struct<order_item_id: long,
                                            order_item_product_id: long,
                                            order_item_quantity: long,
                                            order_item_product_price: float,
                                            order_item_subtotal: float>>"""

In [13]:
orders_df = spark \
.readStream \
.format("json") \
.schema(orders_schema) \
.option("path","data/json_output_result1") \
.load()

In [14]:
orders_df.createOrReplaceTempView("orders")

In [7]:
exploded_orders = spark.sql("""select order_id, customer_id, city, state,
                                pincode, explode(line_items) lines 
                                from orders""")

In [8]:
exploded_orders.createOrReplaceTempView("exploded_orders")

In [9]:
flattened_orders = spark.sql("""select order_id, customer_id, city, state, pincode, 
                                lines.order_item_id as item_id, 
                                lines.order_item_product_id as product_id,
                                lines.order_item_quantity as quantity, 
                                lines.order_item_product_price as price,
                                lines.order_item_subtotal as subtotal 
                                from exploded_orders""")

In [10]:
flattened_orders.createOrReplaceTempView("orders_flattened")

In [11]:
aggregated_orders = spark.sql("""select customer_id, approx_count_distinct(order_id) as orders_placed, 
                                count(item_id) as products_purchased, sum(subtotal) as amount_spent 
                                from orders_flattened 
                                group by customer_id""")

In [ ]:
spark.sql("drop table orders_result1")

In [32]:
streaming_query = aggregated_orders \
.writeStream \
.format("parquet") \
.outputMode("complete") \
.option("checkpointLocation","checkpointdir1") \
.toTable("orders_result1")

AnalysisException: Data source parquet does not support Complete output mode.

In [ ]:
spark.sql("select * from orders_result1").show(5)

In [ ]:
streaming_query.stop()